# Traits

[Pierre-Antoine Champin](http://champin.net/)

http://github.com/pchampin/rust-w3c-2022

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

# Introduction

Traits are similar to Java interfaces, in that

* they can define a number of abstract methods,
* they can be implemented independantly by multiple types.


In [2]:
pub trait Animal {
    fn color(&self) -> &str;
    fn cry(&self) -> &str;
}

NB: trait methods are always considered public

## Trait implementation by a type

In [3]:
pub struct Crow;

impl Animal for Crow {
    fn color(&self) -> &str {
        "black"
    }
    fn cry(&self) -> &str {
        "croak"
    }
}

In [4]:
pub struct Dog { name: String, color: String }

impl Animal for Dog {
    fn color(&self) -> &str {
        &self.color
    }
    fn cry(&self) -> &str {
        "bark"
    }
}

NB: unlike Java interfaces, Rust traits can contain other things than functions:

* constants
* types

## Trait inheritance

Some traits may require one or several super-traits.

Any type implementing this trait **must** also implement all super-traits.

In [5]:
pub trait Pet: Animal {
    fn name(&self) -> &str;
}

impl Pet for Dog {
    fn name(&self) -> &str {
        &self.name
    }
}

## Default implementation

The trait may provide a default implementation of some of its methods.

This default implementation can only depend on other methods of the trait (or of super-traits).


In [6]:
pub trait Rectangle {
    fn width(&self) -> f64;
    fn height(&self) -> f64;
    fn area(&self) -> f64 {
        self.width()*self.height()
    }
    fn perimeter(&self) -> f64 {
        2.0*(self.width()+self.height())
    }
}

## Implementing a trait on foreign types 🦀

A crate can define a new type that implements a foreign trait (i.e. from another crate).

But crate can also define a new trait, and provide implementations of it for foreign types.

In [7]:
pub trait Digits {
    fn digits(&self) -> u32;
}

impl Digits for i32 {
    fn digits(&self) -> u32 {
        let mut x = *self;
        let mut n = 0;
        while x != 0 {
            n += 1;
            x /= 10;
        }
        n
    }
}

12345.digits()

5

## Deriving traits 🦀

Some traits can be *derived* instead of being explicitly implemented\
(i.e. implemented in a standard way, implemented in a special macro).

In [8]:
#[derive(Clone, Copy, Debug, Default)]
pub struct Point {
    x: f64,
    y: f64,
}

let p1 = Point::default(); // p1.x and p1.y will have the default value for f64, i.e. 0.0
let p2 = p1;               // p1 can be copied, since all its fields can be copied
let p3 = p1.clone();       // p3.x and p3.y will be clones of p1.x and p1.x
println!("{:?}", p3);      // a Point is displayed for debug by recursively displaying its fields

Point { x: 0.0, y: 0.0 }


## Using trait methods

To be able to use the methods of a trait, that trait **must** be in the current scope.

In [9]:
{
    //use std::io::Write; // THIS IS REQUIRED
    let mut buffer: Vec<u8> = vec![];
    buffer.write(b"hello");
    buffer
}

Error: the following trait is implemented but not in scope; perhaps add a `use` for it:

# Standard traits

### [`std::clone::Clone`](https://doc.rust-lang.org/std/clone/trait.Clone.html)

Defines the `clone(&self) -> Self` method, allowing to duplicate a value.
*Can be derived.*

NB: example of type that does *not* implement clone: [`std::fs::File`](https://doc.rust-lang.org/std/fs/struct.File.html)

In [10]:
let v1 = vec![1, 2, 3];
let mut v2 = v1.clone();
v2.push(4);
println!("{:?} {:?}", v1, v2);

[1, 2, 3] [1, 2, 3, 4]


NB: `Vec<T>` only implements `Clone` if `T` does.

### [`std::default::Default`](https://doc.rust-lang.org/std/default/trait.Default.html)
    
Defines an associated function `default() -> Self`, constructing a value of this type considered as the "default" value.
*Can be derived.*

In [11]:
i32::default()

0

In [12]:
String::default()

""

In [13]:
Option::<i32>::default()

None

In [14]:
Vec::<i32>::default()

[]

### [`std::cmp::PartialEq`](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html)

Defines a method `eq(&self, other) -> bool` indicating whether `self` and `other` have the same value.
*Can be derived.*

**The `==` operator is syntactic sugar for `PartialEq::eq`.**

The induced relation **must** be symetric and transitive (but not necessarily reflexive, e.g. `NaN`).

There is also a [`PartialOrd`](https://doc.rust-lang.org/std/cmp/trait.PartialOrd.html) trait for operators `<`, `>`, `<=`, `>=`.

### [`std::ops::Add`](https://doc.rust-lang.org/std/ops/trait.Add.html)

Defines a method `add(self, other)` returning the sum of `self` et `other`.

**The `+` operator is syntactic sugar for `Add::add`.**

Il existe aussi les traits [`Sub`](https://doc.rust-lang.org/std/ops/trait.Sub.html) pour l'opérateur `-`,
[`Mul`](https://doc.rust-lang.org/std/ops/trait.Mul.html) pour `*`,
[`Div`](https://doc.rust-lang.org/std/ops/trait.Div.html) pour `/`...

### [`std::fmt::Debug`](https://doc.rust-lang.org/std/fmt/trait.Debug.html)

Defines a method `fmt(&self, ...)` defining how to display `self` for developers.
*Can be derived.*

**The format `{:?}` of `println!` uses `Debug::fmt`.**

There is also a [`Display`](https://doc.rust-lang.org/std/fmt/trait.Display.html) trait targetting end-usres, used by the format `{}`.

### [`std::iter::Iterator`](https://doc.rust-lang.org/std/iter/trait.Iterator.html)

Defines a method `next(&mut self)` returning `Some(v)` (the next value to iter on) or `None` (when it is finished).

**The `for` loop can be used with any type implementing `Iterator`.**

In [15]:
struct CountDown(usize);

impl Iterator for CountDown {
    type Item = usize;
    fn next(&mut self) -> Option<usize> {
        if self.0 > 0 { self.0 -= 1; Some(self.0 + 1) }
        else { None }
    }
}

for i in CountDown(5) { print!("{} ", i) }
println!()

5 4 3 2 1 


()

NB: `Iterator` also has a wealth of other methods with default implementations based on `next`.

In [16]:
CountDown(5).max()

Some(5)

In [17]:
let s: usize = CountDown(5).sum();
s

15

In [18]:
let v: Vec<_> = CountDown(5)
    .skip(1)
    .map(|i| i*i)
    .collect();
v

[16, 9, 4, 1]

### [`std::marker::Copy`](https://doc.rust-lang.org/std/marker/trait.Copy.html)

A *marker* trait has no method, but carries a specific *semantics* for the compiler.

A type implementing `Copy` indicates to the compiler that

* values of this type can be duplicated by a simple memory-copy, and therefore that
* assignment of this type can use the copy semantics rather than the move semantics.

### [`std::ops::Deref`](https://doc.rust-lang.org/std/ops/trait.Deref.html)

Define has an associated type `Target`,
and a method `deref(&self) -> &Self::Target`.

Considering `t: T` implementing `Deref` to a target type `U`,\
the compiler will also automatically call `deref` when:

* `*t` is used, yielding an immutable `U`, or
* `&t` is used in a context where `&U` is expected, or
* `t.method()` is used, where `method` is a method of `U` using `&self`.

There is also a [`DerefMut`](https://doc.rust-lang.org/std/ops/trait.DerefMut.html) trait, for borrowing `Self::Target` mutably.

E.g. `Vec<T>` implements `Deref` and `DerefMut` for `[T]` (slice of T), and therefore "inherits" all its methods.

This is a form of *composition-based* inheritance.

### [`std::ops::Drop`](https://doc.rust-lang.org/std/ops/trait.Drop.html)

Defines a method `drop(&mut self)` method, that the compiler automatically calls when `self` is dropped (de-allocated). 

This is usually not required (the default behaviour is enough),\
except when the type is managing an external resource that needs some clean-up (e.g. file, socket...).

# Traits and genericity

## Polymorphic functions

In [19]:
/// Returns the index of the smallest value in the slice
fn imin<T>(a: &[T]) -> usize 
where
    T: PartialOrd,
{
    let mut im = 0;
    for i in 1..a.len() {
        if a[i] < a[im] {
            im = i;
        }
    }
    im
}

The `where` clause contains **trait bounds**, which constrain which types can be used for `T`\
(only types implementing the given traits).

In return, it allows to use methods of this traits in the function body\
(here: `a[i] < a[im]` thanks to `PartialOrd`).

### Monomorphisation

In [20]:
imin(&[2, -1, 3])

1

In [21]:
imin(&["toto", "titi", "tata"])

2

For each concrete type with which the function `imin` is used,\
the compiler produces a new version of this function.

→ no cost at runtime (except for a bigger code)

### Overriding functions (*alla* Java)

In Java, several functions/methods with the same name can co-exist:
```java
void toto(int i) { System.out.println("You gave me the integer " + i); }
void toto(String s) { System.out.println("You gave me the string " + s); }
```

In Rust, we can use traits to get the same result.

In [22]:
pub trait Description {
    fn description(&self) -> String; 
}

fn toto<D>(d: D)
where
    D: Description
{
    println!("You gave me {}", d.description());
}

In [23]:
impl Description for i32 {
    fn description(&self) -> String {
        format!("the integer {}", self)
    }
}

impl Description for &'_ str {
    fn description(&self) -> String {
        format!("the string {}", self)
    }
}

In [24]:
toto(42);

You gave me the integer 42


Benefit: somebody else may implement the `Description` trait for their own type.

In [25]:
impl Description for Point {
    fn description(&self) -> String {
        format!("the point ({}, {})", self.x, self.y)
    }
}

In [26]:
toto(Point{ x: 1.0, y: 2.5 });

You gave me the point (1, 2.5)


# Epilogue

So, is Rust an object-orientend programming language?

* encapsulation
* abstraction
* polymorphism
* inheritance
* dynamic dispatch